In [1]:
test_semantic_model_url = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/semantic_model.txt' 
questions_url           = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/questions.json'
SF                      = 0.1
max_attempts            = 2
TIMEOUT_SECONDS         = 180  
output_dir              = "/tmp/llm"
model1                  = "o3-mini"
model2                  = "qwen3:30b-a3b"

# Setup

In [2]:
import requests
import json
import duckdb
import pathlib
import time
import re
from   openai import AzureOpenAI, OpenAIError
import datetime
import os
import threading
import pandas as pd


In [3]:
if SF <1 :
 schema = f"DS{str(SF).replace('.', '_')}"
else :
 schema = f'DS{SF:02d}'
os.makedirs(output_dir, exist_ok=True)
db_path = output_dir +"/"+ schema +".duckdb"
if not pathlib.Path(db_path).exists():
    con = duckdb.connect(db_path)
    con.sql("SET memory_limit = '14GB' ")
    con.sql(f"CALL dsdgen(sf={SF})")
    con.close()
con = duckdb.connect()
con.sql(f""" attach '{db_path}' as ds(read_only) ; use ds """)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

In [4]:
def get_ai_response(user_message, LLM, provider, api_key=None, azure_endpoint=None, azure_api_version=None):
    system_prompt = ""
    try:
        github_response = requests.get(test_semantic_model_url)
        github_response.raise_for_status()
        system_prompt = github_response.text.strip()
    except requests.RequestException as e:
        print(f"Error fetching system prompt from GitHub ({test_semantic_model_url}): {e}")
        return f"Error fetching system prompt: {e}"

    generated_text = None
    data = None

    if provider == "ollama":
        url = 'http://127.0.0.1:11434/api/chat'
        headers = {'Content-Type': 'application/json'}
        payload = {
            'model': LLM,
            'messages': [
                {'role': 'system', 'content':"/no_think\n" + system_prompt},
                {'role': 'user', 'content': user_message}
            ],
            'stream': False
        }
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()
            generated_text = data.get('message', {}).get('content', '')
            if generated_text:
                generated_text = generated_text.replace("<think>", "").replace("</think>", "")
        except requests.RequestException as e:
            return f"Error with Ollama API request: {e}"
        except (KeyError, IndexError, AttributeError) as e:
            return f"Unexpected response format from Ollama: {e}\nFull response data: {data}"

    elif provider == "azure_foundry":
        if not azure_endpoint or not azure_api_version or not api_key:
            return "Azure provider requires 'azure_endpoint', 'azure_api_version', and 'api_key' parameters."

        try:
            client = AzureOpenAI(
                api_version=azure_api_version,
                azure_endpoint=azure_endpoint,
                api_key=api_key,
            )

            response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message}
                ],
                model=LLM,
            )

            generated_text = response.choices[0].message.content

        except Exception as e:
            return f"An error occurred with Azure OpenAI: {e}"



    else:
        return f"Unsupported provider for this test setup: {provider}. Supported providers are 'ollama', 'azure_foundry', and 'gemini'."

    if not isinstance(generated_text, str):
         print(f"Generated text is not a string (type: {type(generated_text)}). Cannot clean.")
         if generated_text is None:
              return "Could not retrieve generated text from provider response."
         else:
              return f"Received unexpected output type from {provider}: {type(generated_text)}. Value: {generated_text}"


    cleaned_text = re.sub(r'```(sql|duckdb)?\s*([\s\S]*?)\s*```', r'\2', generated_text, flags=re.IGNORECASE).strip()

    if "```" in cleaned_text or "SELECT " in cleaned_text.upper() or "WITH " in cleaned_text.upper():
         cleaned_text = cleaned_text.replace("```sql", "").replace("```duckdb", "").replace("```", "").strip()


    return cleaned_text

In [5]:
def execute_sql_with_retry(query, test_model, provider, api_key, azure_endpoint, azure_api_version):
    attempt = 1
    current_query = query.strip()

    while attempt <= max_attempts:
        con.sql("SET enable_progress_bar_print = false")
        con.sql("SET progress_bar_time = 0")

        result_container = {"result": None, "error": None}

        def query_thread():
            try:
                result_container["result"] = con.execute(current_query).fetchdf()
            except duckdb.InterruptException:
                result_container["error"] = f"Query interrupted after timeout of {TIMEOUT_SECONDS} seconds."
            except Exception as e:
                result_container["error"] = str(e)

        thread = threading.Thread(target=query_thread)
        thread.start()

        start_time = time.time()
        while thread.is_alive():
            elapsed = time.time() - start_time
            if elapsed > TIMEOUT_SECONDS:
                con.interrupt()
                thread.join()
                return f"Query execution timed out after {TIMEOUT_SECONDS} seconds.", attempt, "query runs forever"
            time.sleep(0.1)

        if result_container["error"]:
            error_message = result_container["error"].lower()
            if "syntax" not in error_message and "parser" not in error_message and "binder" not in error_message:
                return f"Non-syntax error: {result_container['error']}", attempt, current_query

            print(current_query)
            print(f"Attempt {attempt}/{max_attempts} failed with syntax error: {result_container['error']}")

            if attempt == max_attempts:
                return f"Max attempts reached. Last error: {result_container['error']}", attempt, current_query

            message = (
                f"The following SQL query has a syntax error: '{current_query}'.\n"
                f"Error message: {result_container['error']}\n"
                f"Please provide the corrected SQL query. Return only the corrected query without explanation."
            )

            corrected_query = get_ai_response(message, test_model, provider, api_key, azure_endpoint, azure_api_version)

            if corrected_query.startswith("Error"):
                return f"Failed to get corrected query : {corrected_query}", attempt, current_query

            current_query = corrected_query.strip()
            attempt += 1
        else:
            return result_container["result"], attempt, current_query

    return "Unexpected error or loop termination", attempt, current_query


In [6]:
def ask_question(questions, test_model,provider,api_key=None, azure_endpoint=None, azure_api_version=None):
    results_data = [] 
    for i, x in enumerate(questions):
        print(f"Question {i+1}: {x}") # Keep or remove print as needed
        start_time = time.time()
        sql_query_or_error = get_ai_response(x, test_model, provider, api_key, azure_endpoint, azure_api_version)
        print(sql_query_or_error) # Keep or remove print as needed
        query_result_data_json = [] # Initialize as an empty list for JSON result
        attempts_count = None
        error_details = None # Initialize error_details
        if sql_query_or_error is None or sql_query_or_error.startswith("Error"):
            # If get_ai_response returned an error or None, store the error string
            error_message = sql_query_or_error if sql_query_or_error is not None else "AI response was None"
            # print(f"Failed to get query from AI: {error_message}") # Keep or remove print
            # print("Execution: SKIPPED (AI error)") # Keep or remove print
            error_details = f"AI Error: {error_message}" # Store error details
            # query_result_data_json remains empty []
            result_row_count = 0 # Result count is 0 on error
        else:
            result_from_execution, attempts_count,query_returned = execute_sql_with_retry(sql_query_or_error, test_model,provider,api_key, azure_endpoint, azure_api_version)

            display(result_from_execution)
            is_successful = isinstance(result_from_execution, pd.DataFrame)

            if is_successful:
                print("Execution: SUCCESS") # Keep or remove print
                query_result_data_json = result_from_execution.to_dict('records')
                error_details = None # No error details on success
                result_row_count = len(result_from_execution) # Calculate row count
            else:
                print("Execution: FAILED") # Keep or remove print
                #query_result_data_json remains empty []
                error_details = f"Execution Error: {result_from_execution}" # Store error details
                result_row_count = 0 # Result count is 0 on failure


        end_time = time.time()
        duration = round(end_time - start_time, 2)
        print(f"\nExecution Time: {duration:.2f} seconds") # Keep or remove print
        print(f" ############################### ") # Keep or remove print
        results_data.append({
            "model" : test_model,
            "SF" : SF,
            "timestamp": timestamp,
            "nbr": i + 1,
            "question": x,
            "duration_s": duration, 
            "sql_query": query_returned, 
            "attempts": attempts_count,
            "result": query_result_data_json, 
            "result_count": result_row_count, 
            "error_details": error_details 
        })
    log_dir = output_dir +"/log"
    os.makedirs(log_dir, exist_ok=True)
    sanitized_model = re.sub(r'[\\/*?:"<>|]', '_', test_model)
    output_filename = f"{timestamp}_{sanitized_model}.json"
    output_path = os.path.join(log_dir, output_filename)
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=4)
            f.flush()
            os.fsync(f.fileno())
        return f"Successfully processed {len(questions)} questions. Results saved to {output_path}"
    except IOError as e:
        return f"Error saving results to {output_path}: {e}"
    except Exception as e:
        return f"An unexpected error occurred during file saving: {e}"

In [7]:
def display_side_by_side(nbr, model1, model2):
    print(f"question {nbr} : " + duckdb.sql(f" select question from results_filtered where nbr = {nbr}  ").fetchone()[0])
    try:
        sql_query1 = duckdb.sql(f""" select sql_query from results_filtered where nbr = {nbr} and model = '{model1}' """).fetchone()[0]
        sql_query2 = duckdb.sql(f""" select sql_query from results_filtered where nbr = {nbr} and model = '{model2}' """).fetchone()[0]
        df1 = con.sql(sql_query1).df()        
        df2 = con.sql(sql_query2).df() 
        side_by_side = pd.concat([df1, df2], axis=1, keys=[model1, model2])
        display(side_by_side)
        # Print the SQL queries clearly labeled
        print(f"{model1} SQL Query:")
        print("--------------------")
        print(sql_query1)
        print("\n") # Add a newline for separation

        print(f"{model2} SQL Query:")
        print("--------------------")
        print(sql_query2)
        print("\n") # Add a newline for separation
    except Exception as e:
        print(f"Error executing query for nbr {nbr}: {e}")

In [8]:
# get the questions from GitHub
try:
    response = requests.get(questions_url)
    response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)
    questions = json.loads(response.text)

    print("Successfully retrieved questions:")
except requests.exceptions.RequestException as e:
    print(f"Error retrieving file from GitHub: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

Successfully retrieved questions:


# Run Tests

In [9]:
%%time
#ask_question(questions,"o3-mini","azure_foundry",os.getenv("o3_mini_key"), os.getenv("llm_endpoint"),"2025-01-01-preview" )

CPU times: total: 0 ns
Wall time: 0 ns


In [ ]:
%%time
#ask_question(questions,model2,"ollama")

Question 1: What is the overall total sales revenue?
-- What is the overall total sales revenue?
SELECT
  SUM(store_sales.ss_sales_price * store_sales.ss_quantity) AS total_sales
FROM store_sales;


,total_sales
0,5.143009e+08


Execution: SUCCESS

Execution Time: 47.78 seconds
 ############################### 
Question 2: What is the total number of items sold across all transactions?
-- What is the total number of items sold across all transactions?
SELECT SUM(store_sales.ss_quantity) AS total_quantity FROM store_sales;


,total_quantity
0,13939520.0


Execution: SUCCESS

Execution Time: 7.82 seconds
 ############################### 
Question 3: What is the total monetary value of all returned items?
-- What is the total monetary value of all returned items?
SELECT SUM(store_returns.sr_return_amt) AS total_returns FROM store_returns;


...

,item_class,preferred_customer,avg_net_sales
0,country,N,-247.39
1,accent,N,NaN
2,accent,Y,NaN
3,accent,None,NaN
4,accessories,None,NaN
...,...,...,...
295,womens watch,Y,NaN
296,womens watch,N,NaN
297,None,None,NaN
298,None,Y,NaN


Execution: SUCCESS

Execution Time: 59.12 seconds
 ############################### 
Question 20: show all stores net sales , average net sales and percentage of net sales compared to average net sales for each store, ordered by percentage of net sales descending.
-- show all stores net sales , average net sales and percentage of net sales compared to average net sales for each store, ordered by percentage of net sales descending
WITH store_sales_agg AS (
    SELECT
        st.s_store_name,
        SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
    FROM store_sales AS ss
    INNER JOIN store AS st ON ss.ss_store_sk = st.s_store_sk
    GROUP BY st.s_store_name
), store_returns_agg AS (
    SELECT
        st.s_store_name,
        SUM(sr.sr_return_amt) AS total_returns
    FROM store_returns AS sr
    INNER JOIN store AS st ON sr.sr_store_sk = st.s_store_sk
    GROUP BY st.s_store_name
), net_sales_per_store AS (
    SELECT
        COALESCE(ss.s_store_name, sr.s_store_name) AS stor

,store_name,net_sales,average_net_sales,percentage_of_average_net_sales
0,ought,481787676.8,481787676.8,100.0


Execution: SUCCESS

Execution Time: 82.39 seconds
 ############################### 
CPU times: total: 1.67 s
Wall time: 17min 52s


'Successfully processed 20 questions. Results saved to /tmp/llm/log\\20250516_120953_qwen3_30b-a3b.json'

# Check Results for Baseline Model

In [11]:
duckdb.sql(f""" select *,
           cardinality(result[1]) as nbr_columns,
            [result_count , nbr_columns] as result_shape,
            hash(flatten(array_transform(result, x -> json_extract(json(x), '$.*'))) )  as output
            from read_json_auto('{output_dir}/log/*.json') where model in ('{model1}','{model2}') and SF = '{SF}' """).to_view("results")
try:
 duckdb.sql(f""" install excel ; load excel ;copy results to '{output_dir}/test.xlsx' (format 'xlsx', header 'true', overwrite)  """)
except Exception as e:
    print(f"Error exporting to Excel: {e}")
    print("close the excel file or try again.")
# check number of rows and column returned by each model, Null means SQL error
duckdb.sql(f""" 
           with zzzz as (select nbr,question,model,output , timestamp from results )
           pivot( select nbr,model,count(distinct(output)) as result_shape , count(distinct(timestamp)) as nbr_runs from zzzz group by all) 
           on model using min(result_shape) as resultsets ,min(nbr_runs) as nbr_runs order by nbr
        """).show(max_width=120)

┌───────┬────────────────────┬──────────────────┬──────────────────────────┬────────────────────────┐
│  nbr  │ o3-mini_resultsets │ o3-mini_nbr_runs │ qwen3:30b-a3b_resultsets │ qwen3:30b-a3b_nbr_runs │
│ int64 │       int64        │      int64       │          int64           │         int64          │
├───────┼────────────────────┼──────────────────┼──────────────────────────┼────────────────────────┤
│     1 │                  1 │                7 │                        2 │                      9 │
│     2 │                  1 │                7 │                        1 │                      8 │
│     3 │                  1 │                7 │                        1 │                      8 │
│     4 │                  1 │                7 │                        1 │                      8 │
│     5 │                  1 │                7 │                        3 │                      8 │
│     6 │                  1 │                7 │                        1 │      

In [12]:
Model_to_use = model1

In [13]:
df = duckdb.sql(f"""
    with xxx as (
        select nbr, output, timestamp
        from results
        where model = '{Model_to_use}'
        qualify count(distinct output) over(partition by nbr) > 1
        order by nbr
    )
    select nbr, output, min(timestamp) as timestamp
    from xxx
    group by all
    order by nbr
""").df()

# Iterate through each unique nbr
for nbr in df['nbr'].unique():
    # Print the question for the current nbr
    question = duckdb.sql(f"select question from results where nbr = {nbr}").fetchone()[0]
    print(f"Question {nbr}: {question}")
    
    # Get the timestamps and outputs for the current nbr from the query
    nbr_rows = df[df['nbr'] == nbr][['output', 'timestamp']].sort_values(by='timestamp', ascending=False)
    
    # Collect DataFrames for each timestamp
    dfs = []
    labels = []
    for _, row in nbr_rows.iterrows():
        timestamp = row['timestamp']
        try:
            # Get the SQL query for the given nbr, model, and timestamp
            sql_query = duckdb.sql(f"""
                select sql_query
                from results
                where nbr = {nbr}
                and model = '{Model_to_use}'
                and timestamp = '{timestamp}'
            """).fetchone()[0]
            
            # Execute the SQL query and convert to DataFrame
            df_temp = con.sql(sql_query).df()
            dfs.append(df_temp)
            labels.append(f"Timestamp: {timestamp}")
        except Exception as e:
            print(f"Error executing query for nbr {nbr}, timestamp {timestamp}: {e}")
    
    # Display DataFrames side by side
    if dfs:
        try:
            # Concatenate DataFrames horizontally
            combined_df = pd.concat(dfs, axis=1, keys=labels)
            display(combined_df)
        except Exception as e:
            print(f"Error combining DataFrames for nbr {nbr}: {e}")
    else:
        print(f"No valid DataFrames to display for nbr {nbr}")

Question 12: Calculate the return rate for each item category, ordered alphabetically by item category name.


Timestamp: 20250512_201025                                         \
                item_category  total_sales total_returns return_rate   
0                       Books  56231614.81    2903525.42    5.163511   
1                    Children  48923440.85    2398817.12    4.903206   
2                 Electronics  56084975.64    2918402.71    5.203537   
3                        Home  48874282.69    2501742.06    5.118729   
4                     Jewelry  52983266.97    2714389.19    5.123106   
5                         Men  50692116.25    2702402.68    5.331012   
6                       Music  52783919.32    2827889.58    5.357483   
7                       Shoes  47807842.49    2459773.92    5.145126   
8                      Sports  43870221.97    2320822.09    5.290199   
9                       Women  54383250.42    2853326.02    5.246700   
10                       None   1665936.31          0.00    0.000000   
11                       None         0.00     105709.98         NaN   

   Timestamp: 20250512_193907                                         \
                item_category  total_sales total_returns return_rate   
0                       Books  56231614.81    2903525.42    5.163511   
1                    Children  48923440.85    2398817.12    4.903206   
2                 Electronics  56084975.64    2918402.71    5.203537   
3                        Home  48874282.69    2501742.06    5.118729   
4                     Jewelry  52983266.97    2714389.19    5.123106   
5                         Men  50692116.25    2702402.68    5.331012   
6                       Music  52783919.32    2827889.58    5.357483   
7                       Shoes  47807842.49    2459773.92    5.145126   
8                      Sports  43870221.97    2320822.09    5.290199   
9                       Women  54383250.42    2853326.02    5.246700   
10                       None   1665936.31          0.00    0.000000   
11                       None         0.00     105709.98         NaN   

   Timestamp: 20250512_190159              
                item_category return_rate  
0                       Books    5.163511  
1                    Children    4.903206  
2                 Electronics    5.203537  
3                        Home    5.118729  
4                     Jewelry    5.123106  
5                         Men    5.331012  
6                       Music    5.357483  
7                       Shoes    5.145126  
8                      Sports    5.290199  
9                       Women    5.246700  
10                       None         NaN  
11                       None    0.000000

Question 13: What is the monthly trend of net sales during the year 2001, ordere by net sales.


Timestamp: 20250512_221139              Timestamp: 20250512_190159  \
                        month    net_sales                      month   
0                          12  17167258.39                          6   
1                          11  16207294.58                          2   
2                           9  11365462.90                          4   
3                          10  10717099.39                          7   
4                           8  10489886.43                          3   
5                           1   5177088.03                          5   
6                           5   4686864.37                          1   
7                           3   4436523.60                          8   
8                           7   4418270.78                         10   
9                           4   4336369.48                          9   
10                          2   4033130.36                         11   
11                          6   3909591.47                         12   

                 
      net_sales  
0    3909591.47  
1    4033130.36  
2    4336369.48  
3    4418270.78  
4    4436523.60  
5    4686864.37  
6    5177088.03  
7   10489886.43  
8   10717099.39  
9   11365462.90  
10  16207294.58  
11  17167258.39

Question 14: Which customer birth country exhibits the highest average return rate Order by average return rate descending ?


...

Timestamp: 20250515_100245                            \
                  store_name year_1 year_2  pct_change   
0                      ought   2002   2003 -101.420499   
1                      ought   1998   1999   -4.823786   
2                      ought   2001   2002   -1.822518   
3                      ought   2000   2001   -0.988488   
4                      ought   1999   2000    3.784248   

  Timestamp: 20250513_110605                                            \
                  store_name previous_year current_year percent_change   
0                      ought        2002.0       2003.0    -101.420499   
1                        NaN           NaN          NaN            NaN   
2                        NaN           NaN          NaN            NaN   
3                        NaN           NaN          NaN            NaN   
4                        NaN           NaN          NaN            NaN   

  Timestamp: 20250512_221139               ... Timestamp: 20250512_201025  \
                s_store_name current_year  ...               current_year   
0                      ought         2001  ...                     2003.0   
1                      ought         2002  ...                        NaN   
2                      ought         2000  ...                        NaN   
3                      ought         1999  ...                        NaN   
4                      ought         1998  ...                        NaN   

                                  Timestamp: 20250512_193907  \
  previous_year percentage_change                 store_name   
0        2002.0       -101.420499                      ought   
1           NaN               NaN                        NaN   
2           NaN               NaN                        NaN   
3           NaN               NaN                        NaN   
4           NaN               NaN                        NaN   

                          Timestamp: 20250512_190159               \
  pct_change_in_net_sales               s_store_name current_year   
0               -1.822518                      ought         2000   
1                     NaN                      ought         1999   
2                     NaN                      ought         2001   
3                     NaN                      ought         2002   
4                     NaN                      ought         2003   

                                                                 
     net_sales previous_year_net_sales percent_change_net_sales  
0  97320871.87             93772296.02                 3.784248  
1  93772296.02             98524927.98                -4.823786  
2  96358866.54             97320871.87                -0.988488  
3  94602708.88             96358866.54                -1.822518  
4  -1343830.40             94602708.88              -101.420499  

[5 rows x 23 columns]

Question 17: What is the return rate for items sold on weekends versus weekdays, broken down by customer age groups (e.g., under 30, 30-45, over 45), ordered by age group.


Timestamp: 20250515_100245                                        \
                    sale_day age_group   total_sales total_returns   
0                          N     30-45  8.222361e+07    4280256.91   
1                          Y     30-45  3.225223e+07    1553206.42   
2                          N   over 45  1.565045e+08    8147779.85   
3                          Y   over 45  6.436525e+07    3224625.39   
4                          N  under 30  1.096806e+08    5516920.95   
5                          Y  under 30  4.428833e+07    2325349.37   
6                        NaN       NaN           NaN           NaN   
7                        NaN       NaN           NaN           NaN   

              Timestamp: 20250513_160751                          \
  return_rate                   sale_day age_group   total_sales   
0    5.205630                    Weekday     30-45  8.222361e+07   
1    4.815810                    Weekend     30-45  3.225223e+07   
2    5.206100                    Weekday      < 30  1.096806e+08   
3    5.009885                    Weekend      < 30  4.428833e+07   
4    5.029986                    Weekday   Unknown  1.105068e+07   
5    5.250479                    Weekend   Unknown  4.671173e+06   
6         NaN                    Weekday   over 45  1.565045e+08   
7         NaN                    Weekend   over 45  6.436525e+07   

                             ... Timestamp: 20250512_193907           \
  total_returns return_rate  ...                  age_group sale_day   
0    4280256.91    5.205630  ...                      30-45  Weekday   
1    1553206.42    4.815810  ...                      30-45  Weekend   
2    5516920.95    5.029986  ...                    Over 45  Weekday   
3    2325349.37    5.250479  ...                    Over 45  Weekend   
4     607655.67    5.498809  ...                   Under 30  Weekday   
5     200224.04    4.286376  ...                   Under 30  Weekend   
6    8147779.85    5.206100  ...                        NaN      NaN   
7    3224625.39    5.009885  ...                        NaN      NaN   

                                          Timestamp: 20250512_190159           \
    total_sales total_returns return_rate                  age_group sale_day   
0  8.222361e+07    4280256.91    5.205630                      30-45  weekday   
1  3.225223e+07    1553206.42    4.815810                      30-45  weekend   
2  1.675552e+08    8755435.52    5.225405                    over 45  weekday   
3  6.903643e+07    3424849.43    4.960931                    over 45  weekend   
4  1.096806e+08    5516920.95    5.029986                   under 30  weekday   
5  4.428833e+07    2325349.37    5.250479                   under 30  weekend   
6           NaN           NaN         NaN                        NaN      NaN   
7           NaN           NaN         NaN                        NaN      NaN   

                                           
    total_sales total_returns return_rate  
0  8.222361e+07    4280256.91    5.205630  
1  3.225223e+07    1553206.42    4.815810  
2  1.675552e+08    8755435.52    5.225405  
3  6.903643e+07    3424849.43    4.960931  
4  1.096806e+08    5516920.95    5.029986  
5  4.428833e+07    2325349.37    5.250479  
6           NaN           NaN         NaN  
7           NaN           NaN         NaN  

[8 rows x 35 columns]

Question 18: Which item brand has shown the largest decrease in its return rate when comparing the average rate of 2001 to 2002, specifically for stores located in the 'TN' state? (Order by the decrease in return rate descending, and then by brand name alphabetically for ties).


Timestamp: 20250512_221139               Timestamp: 20250512_201025  \
                    item_brand rate_decrease                    i_brand   
0         exportiunivamalg #12     21.075982       exportiamalgamalg #5   
1           brandunivamalg #11     15.292578        exportiunivamalg #3   
2         exportiamalgamalg #5     13.072724          maxiunivamalg #13   
3          exportiunivamalg #3     11.829555           corpunivamalg #7   
4            maxiunivamalg #15     10.735027             scholarmaxi #8   
..                         ...           ...                        ...   
446           edu packbrand #6    -12.034688                        NaN   
447      importoamalgamalg #11    -12.267339                        NaN   
448           edu packbrand #7    -12.906471                        NaN   
449         amalgunivamalg #10    -13.272667                        NaN   
450         corpamalgamalg #16    -13.579079                        NaN   

                  Timestamp: 20250512_193907                          \
    rate_decrease                      brand decrease_in_return_rate   
0       13.072724       exportiamalgamalg #5               13.072724   
1       11.829555        exportiunivamalg #3               11.829555   
2        7.749930          maxiunivamalg #13                7.749930   
3        7.580091           corpunivamalg #7                7.580091   
4        7.050572             scholarmaxi #8                7.050572   
..            ...                        ...                     ...   
446           NaN                        NaN                     NaN   
447           NaN                        NaN                     NaN   
448           NaN                        NaN                     NaN   
449           NaN                        NaN                     NaN   
450           NaN                        NaN                     NaN   

    Timestamp: 20250512_190159                                    \
                    item_brand return_rate_2001 return_rate_2002   
0         exportiamalgamalg #5        13.916474         0.843750   
1          exportiunivamalg #3        18.199075         6.369521   
2            maxiunivamalg #13         9.189426         1.439496   
3             corpunivamalg #7         8.067342         0.487251   
4               scholarmaxi #8         7.526645         0.476072   
..                         ...              ...              ...   
446                        NaN              NaN              NaN   
447                        NaN              NaN              NaN   
448                        NaN              NaN              NaN   
449                        NaN              NaN              NaN   
450                        NaN              NaN              NaN   

                             
    decrease_in_return_rate  
0                 13.072724  
1                 11.829555  
2                  7.749930  
3                  7.580091  
4                  7.050572  
..                      ...  
446                     NaN  
447                     NaN  
448                     NaN  
449                     NaN  
450                     NaN  

[451 rows x 10 columns]

# Compare the Last run of the two Models

In [14]:
duckdb.sql(f""" 
        create or replace temp table results_filtered as
           from results  where model = '{model1}' and timestamp = (select max(timestamp) from results where model = '{model1}') 
           union all 
           from results  where model = '{model2}' and timestamp = (select max(timestamp) from results where model = '{model2}')
""")
# execusion time in seconds
duckdb.sql(f""" 
           pivot( select nbr as question ,model,result_shape , attempts from results_filtered ) 
           on model using min(result_shape) as result,  min(attempts) as attempts order by question
        """).show(max_width=130)

┌──────────┬────────────────┬──────────────────┬──────────────────────┬────────────────────────┐
│ question │ o3-mini_result │ o3-mini_attempts │ qwen3:30b-a3b_result │ qwen3:30b-a3b_attempts │
│  int64   │    int128[]    │      int64       │       int128[]       │         int64          │
├──────────┼────────────────┼──────────────────┼──────────────────────┼────────────────────────┤
│        1 │ [1, 1]         │                1 │ [1, 1]               │                      1 │
│        2 │ [1, 1]         │                1 │ [1, 1]               │                      1 │
│        3 │ [1, 1]         │                1 │ [1, 1]               │                      1 │
│        4 │ [1, 1]         │                1 │ [1, 1]               │                      1 │
│        5 │ [11, 1]        │                1 │ [11, 1]              │                      1 │
│        6 │ [6, 2]         │                1 │ [6, 2]               │                      1 │
│        7 │ [1, 2]         │ 

In [15]:
different_results = duckdb.sql(f"""
    SELECT 
        distinct(nbr) 
    FROM results_filtered 
    WHERE nbr in
    (
    select nbr from results_filtered 
    group by all
    having count(distinct(output)) >  1 
    )
    order by nbr
""").df()['nbr'].tolist()
print(f"Different results for {len(different_results)} questions: {different_results}")
good_results = list(set(list(range(1, 21))) - set(different_results))
print(f"Exact resultsets for {len(good_results)} questions: {good_results}")


Different results for 4 questions: [16, 17, 18, 19]
Exact resultsets for 16 questions: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20]


In [16]:
for nbr in different_results:
    display_side_by_side(nbr, model1, model2)

question 16 : For each store, what was the percentage change in net sales between two consecutive recent years (e.g., 2001 and 2002), ordered alphabetically by store name.


o3-mini                           qwen3:30b-a3b                          \
  store_name year_1 year_2  pct_change  s_store_name year_1 net_sales_year_1   
0      ought   2002   2003 -101.420499           NaN    NaN              NaN   
1      ought   2001   2002   -1.822518           NaN    NaN              NaN   
2      ought   2000   2001   -0.988488           NaN    NaN              NaN   
3      ought   1998   1999   -4.823786           NaN    NaN              NaN   
4      ought   1999   2000    3.784248           NaN    NaN              NaN   

                                             
  year_2 net_sales_year_2 percentage_change  
0    NaN              NaN               NaN  
1    NaN              NaN               NaN  
2    NaN              NaN               NaN  
3    NaN              NaN               NaN  
4    NaN              NaN               NaN

o3-mini SQL Query:
--------------------
-- For each store, what was the percentage change in net sales between two consecutive recent years (e.g., 2001 and 2002), ordered alphabetically by store name.
WITH sales_year AS (
    SELECT
        st.s_store_name,
        d.d_year,
        SUM(ss.ss_sales_price * ss.ss_quantity) AS sales
    FROM store_sales AS ss
    JOIN store AS st
      ON ss.ss_store_sk = st.s_store_sk
    JOIN date_dim AS d
      ON ss.ss_sold_date_sk = d.d_date_sk
    GROUP BY st.s_store_name, d.d_year
),
returns_year AS (
    SELECT
        st.s_store_name,
        d.d_year,
        SUM(sr.sr_return_amt) AS returns
    FROM store_returns AS sr
    JOIN store AS st
      ON sr.sr_store_sk = st.s_store_sk
    JOIN date_dim AS d
      ON sr.sr_returned_date_sk = d.d_date_sk
    GROUP BY st.s_store_name, d.d_year
),
net_sales_per_year AS (
    SELECT
        sy.s_store_name,
        sy.d_year,
        sy.sales - COALESCE(ry.returns, 0) AS net_sales
    FROM sales_year AS 

o3-mini                                                   qwen3:30b-a3b  \
  sale_day age_group   total_sales total_returns return_rate     age_group   
0        N     30-45  8.222361e+07    4280256.91    5.205630         30-45   
1        Y     30-45  3.225223e+07    1553206.42    4.815810         30-45   
2        N   over 45  1.565045e+08    8147779.85    5.206100       Over 45   
3        Y   over 45  6.436525e+07    3224625.39    5.009885       Over 45   
4        N  under 30  1.096806e+08    5516920.95    5.029986      Under 30   
5        Y  under 30  4.428833e+07    2325349.37    5.250479      Under 30   

                                                    
  day_type   total_sales total_returns return_rate  
0  Weekday  8.222361e+07    4280256.91    5.205630  
1  Weekend  3.225223e+07    1553206.42    4.815810  
2  Weekday  1.565045e+08    8147779.85    5.206100  
3  Weekend  6.436525e+07    3224625.39    5.009885  
4  Weekday  1.096806e+08    5516920.95    5.029986  
5  Weekend  4.428833e+07    2325349.37    5.250479

o3-mini SQL Query:
--------------------
-- What is the return rate for items sold on weekends versus weekdays, broken down by customer age groups (under 30, 30-45, over 45), ordered by age group.
WITH SalesAgg AS (
    SELECT
        d.d_weekend AS sale_day, 
        CASE
            WHEN (d.d_year - c.c_birth_year) < 30 THEN 'under 30'
            WHEN (d.d_year - c.c_birth_year) BETWEEN 30 AND 45 THEN '30-45'
            ELSE 'over 45'
        END AS age_group,
        SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
    FROM store_sales AS ss
    JOIN date_dim AS d
      ON ss.ss_sold_date_sk = d.d_date_sk
    JOIN customer AS c
      ON ss.ss_customer_sk = c.c_customer_sk
    WHERE c.c_birth_year IS NOT NULL AND d.d_year IS NOT NULL
    GROUP BY d.d_weekend,
             CASE
                WHEN (d.d_year - c.c_birth_year) < 30 THEN 'under 30'
                WHEN (d.d_year - c.c_birth_year) BETWEEN 30 AND 45 THEN '30-45'
                ELSE 'over 45'
             END
),
Re

o3-mini                    qwen3:30b-a3b                   \
                     brand   decrease               i_brand return_rate_2001   
0     exportiunivamalg #12  21.075982  exportiunivamalg #12        21.075982   
1       brandunivamalg #11  15.292578    brandunivamalg #11        15.292578   
2     exportiamalgamalg #5  13.072724  exportiamalgamalg #5        13.916474   
3      exportiunivamalg #3  11.829555   exportiunivamalg #3        18.199075   
4        maxiunivamalg #15  10.735027     maxiunivamalg #15        10.735027   
..                     ...        ...                   ...              ...   
446       edu packbrand #6 -12.034688                   NaN              NaN   
447  importoamalgamalg #11 -12.267339                   NaN              NaN   
448       edu packbrand #7 -12.906471                   NaN              NaN   
449     amalgunivamalg #10 -13.272667                   NaN              NaN   
450     corpamalgamalg #16 -13.579079                   NaN              NaN   

                                           
    return_rate_2002 return_rate_decrease  
0           0.000000            21.075982  
1           0.000000            15.292578  
2           0.843750            13.072724  
3           6.369521            11.829555  
4           0.000000            10.735027  
..               ...                  ...  
446              NaN                  NaN  
447              NaN                  NaN  
448              NaN                  NaN  
449              NaN                  NaN  
450              NaN                  NaN  

[451 rows x 6 columns]

o3-mini SQL Query:
--------------------
-- Which item brand has shown the largest decrease in its return rate when comparing the average rate of 2001 to 2002, 
-- specifically for stores located in the 'TN' state? Order by the decrease in return rate descending, and then by brand name alphabetically for ties.
WITH 
sales_2001 AS (
    SELECT
        i.i_brand,
        SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
    FROM store_sales AS ss
    JOIN date_dim AS d ON ss.ss_sold_date_sk = d.d_date_sk
    JOIN store AS st ON ss.ss_store_sk = st.s_store_sk
    JOIN item AS i ON ss.ss_item_sk = i.i_item_sk
    WHERE d.d_year = 2001
      AND st.s_state = 'TN'
    GROUP BY i.i_brand
),
returns_2001 AS (
    SELECT
        i.i_brand,
        SUM(sr.sr_return_amt) AS total_returns
    FROM store_returns AS sr
    JOIN date_dim AS d ON sr.sr_returned_date_sk = d.d_date_sk
    JOIN store AS st ON sr.sr_store_sk = st.s_store_sk
    JOIN item AS i ON sr.sr_item_sk = i.i_item_sk
    WHERE d

o3-mini                                  qwen3:30b-a3b  \
         item_class preferred_customer avg_net_sales    item_class   
0           earings               None   2532.885873       country   
1            custom               None   2363.180714        accent   
2          monitors               None   2272.177463        accent   
3         self-help               None   2269.849091        accent   
4    entertainments               None   2174.550328   accessories   
..              ...                ...           ...           ...   
295        business               None   1378.109839  womens watch   
296        wireless               None   1293.797018  womens watch   
297     consignment               None   1272.868095          None   
298          hockey               None   1225.556438          None   
299            arts               None   1143.317292          None   

                                      
    preferred_customer avg_net_sales  
0                    N       -247.39  
1                    N           NaN  
2                    Y           NaN  
3                 None           NaN  
4                    Y           NaN  
..                 ...           ...  
295                  Y           NaN  
296                  N           NaN  
297               None           NaN  
298                  Y           NaN  
299                  N           NaN  

[300 rows x 6 columns]

o3-mini SQL Query:
--------------------
-- For each item class, compare the average net sales value per sales transaction between preferred and non-preferred customers. Order by average net sales descending, and then by item class name alphabetically for ties.
WITH sales_agg AS (
  SELECT
    i.i_class AS item_class,
    c.c_preferred_cust_flag AS preferred_customer,
    SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales,
    COUNT(*) AS sales_transactions
  FROM store_sales AS ss
  JOIN customer AS c ON ss.ss_customer_sk = c.c_customer_sk
  JOIN item AS i ON ss.ss_item_sk = i.i_item_sk
  GROUP BY i.i_class, c.c_preferred_cust_flag
),
returns_agg AS (
  SELECT
    i.i_class AS item_class,
    c.c_preferred_cust_flag AS preferred_customer,
    SUM(sr.sr_return_amt) AS total_returns
  FROM store_returns AS sr
  JOIN customer AS c ON sr.sr_customer_sk = c.c_customer_sk
  JOIN item AS i ON sr.sr_item_sk = i.i_item_sk
  GROUP BY i.i_class, c.c_preferred_cust_flag
)
SELECT
  s.item_class